In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#clone comfyui
%cd /content/drive/MyDrive
!git clone https://github.com/comfyanonymous/ComfyUI

/content/drive/MyDrive
Cloning into 'ComfyUI'...
remote: Enumerating objects: 19190, done.
remote: Counting objects: 100% (276/276), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 19190 (delta 217), reused 126 (delta 126), pack-reused 18914 (from 4)
Receiving objects: 100% (19190/19190), 61.91 MiB | 12.58 MiB/s, done.
Resolving deltas: 100% (12853/12853), done.
Updating files: 100% (378/378), done.


In [3]:
#clone custom nodes
%cd /content/drive/MyDrive/ComfyUI/custom_nodes
!git clone https://github.com/city96/ComfyUI-GGUF

/content/drive/MyDrive/ComfyUI/custom_nodes
Cloning into 'ComfyUI-GGUF'...
remote: Enumerating objects: 588, done.
remote: Counting objects: 100% (300/300), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 588 (delta 257), reused 178 (delta 160), pack-reused 288 (from 1)
Receiving objects: 100% (588/588), 167.08 KiB | 886.00 KiB/s, done.
Resolving deltas: 100% (377/377), done.


In [4]:
#clone custom nodes
%cd /content/drive/MyDrive/ComfyUI/custom_nodes
!git clone https://github.com/mit-han-lab/ComfyUI-nunchaku

/content/drive/MyDrive/ComfyUI/custom_nodes
Cloning into 'ComfyUI-nunchaku'...
remote: Enumerating objects: 415, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 415 (delta 107), reused 94 (delta 91), pack-reused 271 (from 1)
Receiving objects: 100% (415/415), 2.68 MiB | 5.68 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [8]:
#download nunchaku 0.2.0 for torch 2.6 py 3.11
%cd /content
!curl -Lo /content/nunchaku-0.2.0%2Btorch2.6-cp311-cp311-linux_x86_64.whl https://huggingface.co/mit-han-lab/nunchaku/resolve/main/nunchaku-0.2.0%2Btorch2.6-cp311-cp311-linux_x86_64.whl

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1190  100  1190    0     0   3583      0 --:--:-- --:--:-- --:--:--  3584
100  108M  100  108M    0     0   118M      0 --:--:-- --:--:-- --:--:--  118M


In [9]:
#install nunchaku
!pip install /content/nunchaku-0.2.0%2Btorch2.6-cp311-cp311-linux_x86_64.whl

Processing ./nunchaku-0.2.0%2Btorch2.6-cp311-cp311-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-n

In [10]:
#install requirements comfyui+nodes
!pip install -r /content/drive/MyDrive/ComfyUI/requirements.txt
!pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-GGUF/requirements.txt
!pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-nunchaku/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.7/32.7 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=7710d254ff25ae68c53fd6df45ed115bc723433c7774451269c531be2742531f
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f08615

In [ ]:
####################################################

In [ ]:
#### DOWNLOAD MODELS AND SAVE IN ComfyUI/models ####

In [ ]:
####################################################

In [ ]:
%cd /content/drive/MyDrive/ComfyUI
#
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

/content/drive/MyDrive/ComfyUI
--2025-04-26 20:53:19--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.4.0/cloudflared-linux-amd64.deb [following]
--2025-04-26 20:53:19--  https://github.com/cloudflare/cloudflared/releases/download/2025.4.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/d7e7703c-c0be-4512-b40f-145c402e03fd?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250426%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250426T205319Z&X-Amz-Expires=300&X-Amz-Signature=470015f45ad337b2eea39638a0805